# Introduction to Correlations and Covariance

### Goals:

1. To develop intuition for correlation with visual examples.
2. To understand how a "covariance matrix" encodes infomation about correlations.
3. To understand the difference between "covariance" and "correlation" and the advantages and disadvantages of each.
4. To deepen our understanding with examples of quantities that are highly correlated and quantities that are uncorrelated.

### Timing

1. Try to finish this notebook in 25-30 minutes. 

### Question and Answer Template

You can go to the link below, and do "file" -> "make a copy" to make yourself a google doc that you can use to fill in the answers to the question in this weeks notebooks.

https://docs.google.com/document/d/1D3O8AjzSNhCOFeKVrSXmRWSKdOFIdSMkdNTPZ-CIdaM/edit?usp=sharing

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import scipy.optimize as optimize
import datetime

### New functions we will use in this module

| Function Name            | What it does |
| - | - |
| plt.figure               | Make a matplotlib figure, useful for making figures with subplots |
| fig.subplots             | Makes subplots for a figure |
| np.cov                   | Compute the covariance matrix between multiple data series |
| np.corrcoef              | Compute the correlation coefficient between multiple data series |

### Variances and covariances:

The **variance** $\sigma^2$ is a measure of the scatter of a quantity:

$\sigma^2 = \frac{\sum_i (x_i - \mu_x)}{n}^2$,

where $x_i$ are the measurements of the quantity $x$, $n$ is the number of measurements, and $\mu_x$ is the mean of the measurements $\mu_x = \frac{\sum_i x_i}{n}$.

The **covariance** is a measure of how variations in one quantity match variations in a second quantity. 

The equation for the covariance is quite similar to the equation for the variance. Suppose that each data point has has two quantities that describe it, $(x_{i},y_{i})$. (In other words, it's a data point in a multi-dimensional space.) Then the equation for the covariance is:

$\sigma_{xy} = \frac{\sum_i (x_i - \mu_x) (y_i - \mu_y)}{n}$

i.e., we replace one of the factors of $(x_i - \mu_x)$ with $(y_i - \mu_y)$.

A couple of things to notice about the covariance:

1. If, for a given pair of values, both $(x_i - \mu_x)$ and $(y_i - \mu_y)$ have the same sign, (i.e., they are both above or below the average), then $(x_i - \mu_x) (y_i - \mu_y)$ will be greater than zero.  Conversely, if they have opposite signs, then $(x_i - \mu_x) (y_i - \mu_y)$ will be less than zero.   Thus, the covariance can be either positive or negative.

2. Pairs of values where the absolute values $|(x_i - \mu_x)|$ and $|(y_i - \mu_y)|$ are both large will contribute a lot to the covariance.  Pairs where one of the values is very close to the mean will contribute very little.

Because the equations for the variance and covariance are so similar, we often compute both the variances and covariances at the same time and put all the results in an array or matrix. This is called the **covariance matrix**:

$$
\begin{bmatrix}
\sigma_{xx} & \sigma_{xy} \\
\sigma_{yx} & \sigma_{yy} \\
\end{bmatrix}
$$

### Let's make a couple of functions to simulate 2D data with different levels of covariances

The `draw_2d_gaussian` function below draws data from a **pair** of Gaussians centered at zero ($\mu_x = \mu_y = 0$).

We need to specify:

- "axes" → plotting option

- "color" → plotting option

- $n$ → number of data points

- $\sigma_{xx}$ → variance of Gaussian for variable x

- $\sigma_{yy}$ → variance of Gaussian for variable y

- $\sigma_{xy}$ → covariance of Gaussian for variables x and y

You can safely ignore the details of the functions, but if you're curious, [here](https://en.wikipedia.org/wiki/Multivariate_normal_distribution#Drawing_values_from_the_distribution) is an explanation of the method of how to specify a multi-dimentional normal distribution with a covariance matrix.  The point here is to visualize the resulting scatter plots and measure the covariances.


In [ ]:
def gen_2d_gaussian(n, sigma_xx, sigma_yy, sigma_xy):
    K_0 = np.array([[sigma_xx, sigma_xy],[sigma_xy, sigma_yy]]) # make a matrix-like object 
    epsilon = 0.0001
    K = K_0 + epsilon*np.identity(2)
    L = np.linalg.cholesky(K)
    u = np.random.normal(size=2*n).reshape(2, n)
    x = np.dot(L, u)
    return x

In [ ]:
def draw_2d_gaussian(axes, color, n, sigma_xx, sigma_yy, sigma_xy):
    vals = gen_2d_gaussian(n, sigma_xx, sigma_yy, sigma_xy)
    axes.set_xlim(-5, 5)
    axes.set_ylim(-5, 5)
    axes.scatter(vals[0], vals[1], color=color)
    axes.set_xlabel("x")
    axes.set_ylabel("y")
    return vals

### Effect of changing the covariance, $\sigma_{xy}$.

Let's make 4 plots with unit variances and with 4 different values of the covariance.

In [ ]:
fig = plt.figure()
axes = fig.subplots(2,2)
vals_0 = draw_2d_gaussian(axes[0,0], 'r', 1000, 1, 1, 0)
vals_1 = draw_2d_gaussian(axes[0,1], 'g', 1000, 1, 1, 1)
vals_2 = draw_2d_gaussian(axes[1,0], 'b', 1000, 1, 1, -1)
vals_3 = draw_2d_gaussian(axes[1,1], 'y', 1000, 1, 1, 0.5)
plt.show()

In [ ]:
# compute covariance matrix from the generated data
print("Covariance matrix for plot 0,0:\n", np.cov(vals_0))
print("Covariance matrix for plot 0,1:\n", np.cov(vals_1))
print("Covariance matrix for plot 1,0:\n", np.cov(vals_2))
print("Covariance matrix for plot 1,1:\n", np.cov(vals_3))

### Effect of changing the variances, $\sigma_{xx}, \sigma_{yy}$:

Let's make 4 plots with different variances.

In [ ]:
fig = plt.figure()
axes = fig.subplots(2,2)
vals_0 = draw_2d_gaussian(axes[0,0], 'r', 1000, 2, 2, 0)
vals_1 = draw_2d_gaussian(axes[0,1], 'g', 1000, 0.2, 1, 0)
vals_2 = draw_2d_gaussian(axes[1,0], 'b', 1000, 2, 0.2, 0)
vals_3 = draw_2d_gaussian(axes[1,1], 'y', 1000, 0.2, 0.2, 0.18)
plt.show()

In [ ]:
# compute covariance matrix from the generated data
print("Covariance matrix for plot 0,0:\n", np.cov(vals_0))
print("Covariance matrix for plot 0,1:\n", np.cov(vals_1))
print("Covariance matrix for plot 1,0:\n", np.cov(vals_2))
print("Covariance matrix for plot 1,1:\n", np.cov(vals_3))

### Questions for discussion

#### 1.1 Do the plots look how you expected? Explain how the shapes and spread of the four plots relate to the values you see in the covariance matrix. 

### Another way to consider correlations is to ask what part of the variance in one quantity is  tied to the variance of another quantity.

To do this, we want to factor out the variances of the two quantities and compute the "correlation factor" or "correlation coefficient".

$c_{xy} = \frac{\sigma_{xy}}{\sqrt{\sigma_{xx}\sigma_{yy}}}$

Note that $c_{xx} = c_{yy} = 1$, i.e., each quantity is 100% correlated with itself.

We can again put together $c_{xx}$, $c_{yy}$, $c_{xy}$ in a matrix called the **correlation matrix**:
$$
\begin{bmatrix}
c_{xx} & c_{xy} \\
c_{yx} & c_{yy} \\
\end{bmatrix}
$$

Let's remake the second set of plots and evaluate the correlations.


In [ ]:
fig = plt.figure()
axes = fig.subplots(2,2)
vals_0 = draw_2d_gaussian(axes[0,0], 'r', 1000, 2, 2, 0)
vals_1 = draw_2d_gaussian(axes[0,1], 'g', 1000, 0.2, 1, 0)
vals_2 = draw_2d_gaussian(axes[1,0], 'b', 1000, 2, 0.2, 0)
vals_3 = draw_2d_gaussian(axes[1,1], 'y', 1000, 0.2, 0.2, 0.18)
plt.show()

print("Correlation matrix for plot 0,0:\n", np.corrcoef(vals_0))
print("Correlation matrix for plot 0,1:\n", np.corrcoef(vals_1))
print("Correlation matrix for plot 1,0:\n", np.corrcoef(vals_2))
print("Correlation matrix for plot 1,1:\n", np.corrcoef(vals_3))

### Question for discussion

#### 2.1 What do you see as the advantage / disadvantage of using the correlation coefficient versus the covariance?

## Ok, let's load up the Vela data

This is the same stuff we used to load the data last week.  

In [ ]:
data = np.loadtxt(open("../data/Vela_Flux.txt", 'rb'), usecols=range(7))

# This is how we pull out the data from columns in the array.

# This is the date in "Mission Elapsed Time"
# For the Fermi mission, this is defined to be the number of seconds since the start of 2001.
date_MET = data[:,0]
# This is the offset in seconds between the Fermi "MET" and the UNIX "epoch" used by matplotlib
MET_To_Unix = 978336000

# These are the numbers of photons observed from Vela each week in the "low" Energy Band (100 MeV - 800 MeV)
nObs_LE = data[:,1]

# These are the number of photons expected from Vela each week, under the assumption that it is 
# not varying at all, and the only differences depend on how long we spent looking at Vela
# that particular weeek
nExp_LE = data[:,2]

# These are the band bounds, in MeV
LE_bounds = (100., 800.)

# This is the "significance" of the variation for each week.  We will discuss this more later
signif_LE = data[:,3]

nObs_HE = data[:,4]
nExp_HE = data[:,5]
signif_HE = data[:6]
HE_bounds = (800., 10000.)

# This converts the dates to something that matplotlib understands
dates = [datetime.datetime.fromtimestamp(date + MET_To_Unix) for date in date_MET]

### Example with quantities that are highly correlated.

Last week we saw that for the Vela data the numbers of observed and expected counts where highly correlated.  I.e., that Vela wasn't flaring unexpectedly.

Let's quantify that.

In [ ]:
plt.scatter(nExp_LE, nObs_LE)
plt.xlabel(r"$n_{\rm exp}$ [per week]")
plt.ylabel(r"$n_{\rm obs}$ [per week]")
plt.show()

correl = np.corrcoef(nObs_LE, nExp_LE)
print(f"The expected counts are {correl[0,1]:0.3f} correlated with the observed counts")

### Is there a long-term trend to the Vela flux?

Let's use the correlation to see if there is a long-term trend to the Vela pulsar flux. 

Ok, so let's convert the time axis to a nice small number (i.e., let's scale the time to years, and let's set the zero point to something in the middle of the time range).  This will save us having lots of really big numbers floating around, which would happen if we left the time in seconds since 2001.

In [ ]:
date_YEAR = 2001 +  (date_MET / (24*3600*365))
years_since_mid_2014 = date_YEAR  - 2014.5

In [ ]:
excess_counts = nObs_LE-nExp_LE
plt.scatter(years_since_mid_2014, excess_counts)
plt.xlabel(r"Time since mid 2014 [years]")
plt.ylabel(r"$n_{\rm obs}$ [per week]")
plt.show()

In [ ]:
cov = np.cov(years_since_mid_2014, excess_counts)
correl = np.corrcoef(years_since_mid_2014, excess_counts)

### Let's have a look at the pieces of the covariance matrix

In [ ]:
print(f"The xx element of the covariance matrix is {cov[0,0]:.2f} years**2")
print(f"The standard deviation of the x element (i.e., the year) is {np.sqrt(cov[0,0]):.2f} years")
print(f"The yy element of the covariance matrix is {cov[1,1]:.2f} counts**2")
print(f"The standard deviation of the yy element (i.e., the excess counts) is {np.sqrt(cov[1,1]):.1f} counts")
print(f"The xy element of the covariance matrix is {cov[0,1]:.2f} years*counts")
print(f"The x-y correlation coefficient is {correl[0,1]:.3f}")

### Question for discussion

#### 3.1 Do you think that the small correlation we found in time vs $n_{\rm obs}$ is going to be statistically significant?  Why or why not?  (Some things you might want to think about: Does the contrast between the last two plots affect your opinion?  What about the difference between the correlation values?)